# Basic use of Trulens on Basic and Advance RAG

In [20]:
import utils

import os
import openai
openai.api_key = ""

In [3]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [4]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.core.schema.Document'>
Doc ID: 7c4b7632-ed9f-493f-b774-a9360dcffb5b
Text: PAGE 1 Founder, DeepLearning.AI Collected Insights from Andrew
Ng How to  Build Your Career in AI A Simple Guide


## Basic RAG pipeline

In [5]:
from llama_index.core import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [6]:
from llama_index.core import DocumentSummaryIndex, Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding


llm= OpenAI(model="gpt-3.5-turbo", temperature=0.1)
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

Settings.llm = llm
Settings.embed_model = embed_model

index = DocumentSummaryIndex.from_documents(
    documents=[document],
    embed_model=embed_model,
    llm=llm
)

/opt/anaconda3/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
current doc id: f2113563-f345-4cc9-9c8c-dde4b279173c


In [7]:
query_engine = index.as_query_engine()

In [8]:
response = query_engine.query(
    "What are steps to take when finding projects to build your experience?"
)
print(str(response))

Identify a business problem, brainstorm AI solutions, determine milestones, assess feasibility and value of potential solutions, budget for resources, and iterate on the project as needed.


## Evaluation setup using TruLens

In [9]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

What are the keys to building a career in AI?
How can teamwork contribute to success in AI?
What is the importance of networking in AI?
What are some good habits to develop for a successful career?
How can altruism be beneficial in building a career?
What is imposter syndrome and how does it relate to AI?
Who are some accomplished individuals who have experienced imposter syndrome?
What is the first step to becoming good at AI?
What are some common challenges in AI?
Is it normal to find parts of AI challenging?


In [10]:
# You can try your own question:
new_question = "What is the right AI job for me?"
eval_questions.append(new_question)

In [11]:
print(eval_questions)

['What are the keys to building a career in AI?', 'How can teamwork contribute to success in AI?', 'What is the importance of networking in AI?', 'What are some good habits to develop for a successful career?', 'How can altruism be beneficial in building a career?', 'What is imposter syndrome and how does it relate to AI?', 'Who are some accomplished individuals who have experienced imposter syndrome?', 'What is the first step to becoming good at AI?', 'What are some common challenges in AI?', 'Is it normal to find parts of AI challenging?', 'What is the right AI job for me?']


In [12]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

/var/folders/_c/_wx3_r550ylgdgq0h4q8sllw0000gn/T/ipykernel_24703/2801267818.py:1: DeprecationWarning: The `trulens_eval` module is deprecated. See https://www.trulens.org/component_guides/other/trulens_eval_migration/ for instructions on migrating to `trulens.*` modules.
  from trulens_eval import Tru


🦑 Initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `TruSession` to prevent this.


Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]


In [13]:
from local_utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.calls[-1].rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.calls[-1].rets.source_nodes[:].node.text.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
instrumenting <class 'llama_index.llms.openai.base.OpenAI'> for base <class 'llama_index.llms.openai.base.OpenAI'>
instrumenting <class 'llama_index.llms.openai.base.OpenAI'> for base <class 'llama_index.core.llms.function_calling.FunctionCallingLLM'>
instrumenting <class 'llama_index.llms.openai.base.OpenAI'> for base <class 'llama_index.core.llms.llm.LLM'>
instrumenting <cla

In [14]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

calling <function BaseQueryEngine.query at 0x1658d2c20> with (<llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine object at 0x1679a38b0>, 'What are the keys to building a career in AI?')
calling <function RetrieverQueryEngine.retrieve at 0x16695d990> with (<llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine object at 0x1679a38b0>, QueryBundle(query_str='What are the keys to building a career in AI?', image_path=None, custom_embedding_strs=None, embedding=None))
calling <function BaseRetriever.retrieve at 0x165c97010> with (<llama_index.core.indices.document_summary.retrievers.DocumentSummaryIndexEmbeddingRetriever object at 0x1679a0e50>, QueryBundle(query_str='What are the keys to building a career in AI?', image_path=None, custom_embedding_strs=None, embedding=None))
calling <function DocumentSummaryIndexEmbeddingRetriever._retrieve at 0x16604dea0> with (<llama_index.core.indices.document_summary.retrievers.DocumentSummaryIndexEmbeddingRet

In [15]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [16]:
import pandas as pd 
pd.set_option('display.max_columns', None)


In [17]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Groundedness,Groundedness_calls,Groundedness feedback cost in USD,Answer Relevance,Answer Relevance_calls,Answer Relevance feedback cost in USD,app_name,app_version,latency,total_tokens,total_cost,cost_currency
0,app_hash_6e8221fde876d15698298cea8c0d1bd6,"{'tru_class_info': {'name': 'TruLlama', 'modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_bf4a96f8ff93980bc3082d22a5bfe93e,What is the right AI job for me?,The right AI job for you is one that aligns wi...,-,{'record_id': 'record_hash_bf4a96f8ff93980bc30...,"{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-01-11T12:57:47.340696"", ""...",2025-01-11T12:58:03.890813,NaN,NaN,NaN,NaN,NaN,NaN,Direct Query Engine,base,16.533941,12794,0.019252,USD
1,app_hash_6e8221fde876d15698298cea8c0d1bd6,"{'tru_class_info': {'name': 'TruLlama', 'modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_3100cc0cf9c18dbc71d7133bfbb4a071,Is it normal to find parts of AI challenging?,"Yes, it is normal to find parts of AI challeng...",-,{'record_id': 'record_hash_3100cc0cf9c18dbc71d...,"{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-01-11T12:57:43.120945"", ""...",2025-01-11T12:57:47.038671,NaN,NaN,NaN,NaN,NaN,NaN,Direct Query Engine,base,3.891627,12733,0.019130,USD
2,app_hash_6e8221fde876d15698298cea8c0d1bd6,"{'tru_class_info': {'name': 'TruLlama', 'modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_420ceb185b0ac9e6179a3c0c22a43eb7,What are some common challenges in AI?,Common challenges in AI include understanding ...,-,{'record_id': 'record_hash_420ceb185b0ac9e6179...,"{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-01-11T12:57:39.882005"", ""...",2025-01-11T12:57:42.843013,1.0,"[{'args': {'source': ['PAGE 1\nFounder, DeepLe...",0.021446,NaN,NaN,NaN,Direct Query Engine,base,2.955518,12708,0.019081,USD
3,app_hash_6e8221fde876d15698298cea8c0d1bd6,"{'tru_class_info': {'name': 'TruLlama', 'modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_814c3a7be9e9990e1042cd320e9ce68a,What is the first step to becoming good at AI?,The first step to becoming good at AI is to em...,-,{'record_id': 'record_hash_814c3a7be9e9990e104...,"{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-01-11T12:57:36.101395"", ""...",2025-01-11T12:57:39.621562,NaN,NaN,NaN,0.333333,[{'args': {'prompt': 'What is the first step t...,0.000130,Direct Query Engine,base,3.505216,12754,0.019171,USD
4,app_hash_6e8221fde876d15698298cea8c0d1bd6,"{'tru_class_info': {'name': 'TruLlama', 'modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_b117f0dc0e2d4eb891412ed06ed2d9dd,Who are some accomplished individuals who have...,"Former Facebook COO Sheryl Sandberg, U.S. firs...",-,{'record_id': 'record_hash_b117f0dc0e2d4eb8914...,"{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-01-11T12:57:28.766718"", ""...",2025-01-11T12:57:35.849730,NaN,NaN,NaN,1.000000,[{'args': {'prompt': 'Who are some accomplishe...,0.000114,Direct Query Engine,base,7.075107,12720,0.019103,USD


In [19]:
print(records.columns)

Index(['app_id', 'app_json', 'type', 'record_id', 'input', 'output', 'tags',
       'record_json', 'cost_json', 'perf_json', 'ts', 'Groundedness',
       'Groundedness_calls', 'Groundedness feedback cost in USD',
       'Answer Relevance', 'Answer Relevance_calls',
       'Answer Relevance feedback cost in USD', 'app_name', 'app_version',
       'latency', 'total_tokens', 'total_cost', 'cost_currency'],
      dtype='object')


In [16]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...


/var/folders/_c/_wx3_r550ylgdgq0h4q8sllw0000gn/T/ipykernel_19978/1997498841.py:2: DeprecationWarning: Method `run_dashboard` has been renamed or moved to `trulens.dashboard.run.run_dashboard`.

  tru.run_dashboard()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.1.134:52934 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## Advanced RAG pipeline

### 1. Sentence Window retrieval

In [17]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [18]:
from local_utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
from local_utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [21]:
window_response = sentence_window_engine.query(
    "how do I get started on a personal project in AI?"
)
print(str(window_response))

Start by selecting a project that interests you and aligns with your current skill level. Clearly define the goals and objectives of the project. Communicate the value of your project idea to others to gather feedback and support. Begin by working on smaller, manageable tasks to build momentum and gradually progress to more complex undertakings. Remember that each project is a step in your learning journey, so don't worry about starting small.


In [22]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = "Sentence Window Query Engine"
)

Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]


instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.base.embeddings.base.BaseEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.schema.TransformComponent'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.schema.BaseComponent'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'pydantic.main.BaseModel'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.instrumentation.DispatcherSpanMixin'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'abc

In [23]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

calling <function BaseQueryEngine.query at 0x14cafd2d0> with (<llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine object at 0x3224f3220>, 'What are the keys to building a career in AI?')
calling <function RetrieverQueryEngine.retrieve at 0x14dc14c10> with (<llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine object at 0x3224f3220>, QueryBundle(query_str='What are the keys to building a career in AI?', image_path=None, custom_embedding_strs=None, embedding=None))
calling <function BaseRetriever.retrieve at 0x14ced8280> with (<llama_index.core.indices.vector_store.retrievers.retriever.VectorIndexRetriever object at 0x32246b1c0>, QueryBundle(query_str='What are the keys to building a career in AI?', image_path=None, custom_embedding_strs=None, embedding=None))
calling <function VectorIndexRetriever._retrieve at 0x14d4f1cf0> with (<llama_index.core.indices.vector_store.retrievers.retriever.VectorIndexRetriever object at 0x32246b1c0>, QueryBundle

In [24]:
tru.get_leaderboard(app_ids=[])

,,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_name,app_version,,,,,
Sentence Window Query Engine,base,0.777778,0.479167,0.506173,2.125207,0.003176


In [25]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Dashboard already running at path:   Network URL: http://192.168.1.134:52934



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### 2. Auto-merging retrieval

In [27]:
from local_utils import build_automerging_index

automerging_index = build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index"
)

In [28]:
from local_utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [29]:
auto_merging_response = automerging_query_engine.query(
    "How do I build a portfolio of AI projects?"
)
print(str(auto_merging_response))

> Merging 1 nodes into parent node.
> Parent node id: 44bccab4-752a-462e-acd1-64b530bc28cf.
> Parent node text: PAGE 21
Building a Portfolio of 
Projects that Shows 
Skill Progression 
CHAPTER 6
PROJECTS

> Merging 1 nodes into parent node.
> Parent node id: f7b68605-564c-4934-9cc1-6f746cd1ab16.
> Parent node text: PAGE 17
Finding Projects that 
Complement Your 
Career Goals
CHAPTER 5
PROJECTS

> Merging 1 nodes into parent node.
> Parent node id: 7b144869-a14e-43f0-a279-c2d9025031a8.
> Parent node text: PAGE 14
Scoping Successful 
AI Projects
CHAPTER 4
PROJECTS

> Merging 1 nodes into parent node.
> Parent node id: 4b29ce6c-c454-4e1f-9000-a34d368ab87f.
> Parent node text: PAGE 21
Building a Portfolio of 
Projects that Shows 
Skill Progression 
CHAPTER 6
PROJECTS

> Merging 1 nodes into parent node.
> Parent node id: 3f4467af-ab3f-4d34-b1d9-a3dc5c241499.
> Parent node text: PAGE 17
Finding Projects that 
Complement Your 
Career Goals
CHAPTER 5
PROJECTS

> Merging 1 nodes into parent no

In [30]:
tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]


instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.base.embeddings.base.BaseEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.schema.TransformComponent'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.schema.BaseComponent'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'pydantic.main.BaseModel'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.instrumentation.DispatcherSpanMixin'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'abc

In [31]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

calling <function BaseQueryEngine.query at 0x14cafd2d0> with (<llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine object at 0x14d4ed5a0>, 'What are the keys to building a career in AI?')
calling <function RetrieverQueryEngine.retrieve at 0x14dc14c10> with (<llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine object at 0x14d4ed5a0>, QueryBundle(query_str='What are the keys to building a career in AI?', image_path=None, custom_embedding_strs=None, embedding=None))
calling <function BaseRetriever.retrieve at 0x14ced8280> with (<llama_index.core.retrievers.auto_merging_retriever.AutoMergingRetriever object at 0x3b2571450>, QueryBundle(query_str='What are the keys to building a career in AI?', image_path=None, custom_embedding_strs=None, embedding=None))
calling <function AutoMergingRetriever._retrieve at 0x3540dd090> with (<llama_index.core.retrievers.auto_merging_retriever.AutoMergingRetriever object at 0x3b2571450>, QueryBundle(query_str='What

In [32]:
tru.get_leaderboard(app_ids=[])

,,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_name,app_version,,,,,
Automerging Query Engine,base,0.9,0.555556,0.566667,2.782513,0.000866


In [33]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Dashboard already running at path:   Network URL: http://192.168.1.134:52934



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>